<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=610d60b3a07d50090ca0a747587812e3e426caa87a49407a0b7e989027b9875a
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.0


    Uninstalling protobuf-6.33.0:
      Successfully uninstalled protobuf-6.33.0


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-11-06 14:42:22
-------------------
qualified stocks: 87
with latest results: 26
still star stocks: 16
-------------------
Initial Investment:  1.26 C
CY Investment:  1.53 C
Reserve:  1.05 L
Current:  1.43 C
-------------------
Today PnL: -1.39 L (-0.96%)
Current PnL: -21.43 L (-14.04%)
CY Booked + Current PnL: -8.78 L (-5.75%)
-------------------
Total profit:  1.92 L
Total loss:  -23.35 L
-------------------
Total Booked + Current PnL: 18.20 L (14.49%)
Total Booked PnL: 39.63 L (31.55%)
Curr Year Booked PnL: 12.65 L (8.84%)
Prev Year Booked PnL: 26.98 L (21.48%)
Est FTT:  2.32 C
Est FTT PnL: 88.41 L (61.77%)
Deployed:  1.26 C
Current:  1.43 C
CAGR/XIRR %: 7.65%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
58,RELIANCE,1533.00,-6.91,67.0,X-LC,5.81,232924.0,22558.0,6219.0,1.36,10.72,2.67,13.68,37.0,3.63,1.64,29.02,XY25,NTT,REFINERIES
79,UNIONBANK,163.00,-5.43,66.0,M-LC,10.67,171732.0,30892.0,13601.0,-0.33,21.93,7.92,31.59,66.0,2.27,1.21,54.23,XY24,NTT,BANKS
78,TTKPRESTIG,770.00,102.56,52.0,M-SC,1.75,87246.0,-13531.0,13628.0,-0.42,-13.43,15.62,0.09,245.0,-0.99,0.61,14.62,OX40N,NTT,DURABLES
2,ABBOTINDIA,35195.00,-10.45,43.0,X-MC,2.60,88530.0,-2148.0,17051.0,0.51,-2.37,19.26,16.44,101.0,-0.13,0.62,17.33,X40,ATH,PHARMA
33,ICICIGI,2252.93,-14.07,56.0,X-MC,5.53,147956.0,11823.0,18761.0,-1.99,8.68,12.68,22.47,91.0,0.63,1.04,22.79,X40,ATH,INSURANCE


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANPAINT,3465.66,-9.27,69.0,X-LC,4.63,229055.0,-22713.0,75932.0,4.67,-9.02,33.15,21.13,27.0,-0.30,1.61,23.21,X40,ATH,PAINTS
24,EASEMYTRIP,26.40,-16.61,55.0,M-SC,27.30,80211.0,-96604.0,177403.0,3.66,-54.64,221.17,45.70,197.0,-0.54,0.56,4.71,XY24,NTT,TRAVEL
67,SIS,528.00,2045.60,50.0,H-SC,2.70,86598.0,-24434.0,48045.0,2.60,-22.01,55.48,21.26,156.0,-0.51,0.61,17.00,OX40N,NTT,MISC
58,RELIANCE,1533.00,-6.91,67.0,X-LC,5.81,232924.0,22558.0,6219.0,1.36,10.72,2.67,13.68,37.0,3.63,1.64,29.02,XY25,NTT,REFINERIES
21,DABUR,735.00,-6.18,60.0,X-MC,2.91,254164.0,6096.0,102301.0,1.35,2.46,40.25,43.70,102.0,0.06,1.79,17.26,XY24,BTT,FMCG


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
64,SFL,1287.00,31.28,45.0,M-SC,4.39,173299.0,-88919.0,158742.0,-6.35,-33.91,91.60,26.63,229.0,-0.56,1.22,7.75,XY24,NTT,MISC
8,ATULAUTO,844.00,3359.46,34.0,M-SC,3.27,155278.0,-41854.0,131676.0,-4.43,-21.23,84.80,45.57,236.0,-0.32,1.09,10.09,XY24,NTT,AUTO
84,VOLTAS,1530.00,-6.95,30.0,X-MC,4.46,195315.0,3573.0,34180.0,-4.22,1.86,17.50,19.69,99.0,0.10,1.37,8.79,XY25,NTT,AC
19,COFFEEDAY,80.00,-49.70,35.0,L-SC,27.74,74880.0,-38669.0,78721.0,-3.63,-34.05,105.13,35.27,268.0,-0.49,0.53,83.27,XR,NTT,HOTELS
46,KPIGREEN,731.05,8.64,58.0,H-SC,7.39,129490.0,4193.0,54735.0,-3.42,3.35,42.27,47.03,141.0,0.08,0.91,62.13,MH,ATH,POWER


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
31,HINDZINC,730.22,24.68,43.0,M-LC,10.43,203493.0,-1583.0,114160.0,-1.17,-0.77,56.1,54.89,52.0,-0.01,1.43,23.14,X5K,ATH,METALS


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,TTKPRESTIG,770.0,102.56,52.0,M-SC,1.75,87246.0,-13531.0,13628.0,-0.42,-13.43,15.62,0.09,245.0,-0.99,0.61,14.62,OX40N,NTT,DURABLES
45,KANSAINER,340.0,-67.61,48.0,H-SC,2.72,223200.0,-46467.0,82807.0,-0.22,-17.23,37.10,13.47,138.0,-0.56,1.57,13.49,XY24,NTT,PAINTS
17,CERA,9475.0,-21.96,52.0,H-SC,1.76,142266.0,-33637.0,75657.0,-1.73,-19.12,53.18,23.89,149.0,-0.44,1.00,22.87,OX40N,NTT,CERAMICS
67,SIS,528.0,2045.60,50.0,H-SC,2.70,86598.0,-24434.0,48045.0,2.60,-22.01,55.48,21.26,156.0,-0.51,0.61,17.00,OX40N,NTT,MISC
42,JCHAC,2282.0,19921.74,63.0,M-SC,1.52,97888.0,-29917.0,29905.0,0.03,-23.41,30.55,-0.01,232.0,-1.00,0.69,10.84,OX40N,NTT,AC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
79,UNIONBANK,163.0,-5.43,66.0,M-LC,10.67,171732.0,30892.0,13601.0,-0.33,21.93,7.92,31.59,66.0,2.27,1.21,54.23,XY24,NTT,BANKS


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
61,SAIL,228.00,50.76,61.0,M-MC,9.97,237791.0,12829.0,154826.0,-0.07,5.70,65.11,74.53,192.0,0.08,1.67,39.80,XY24,BTT,STEEL
46,KPIGREEN,731.05,8.64,58.0,H-SC,7.39,129490.0,4193.0,54735.0,-3.42,3.35,42.27,47.03,141.0,0.08,0.91,62.13,MH,ATH,POWER
79,UNIONBANK,163.00,-5.43,66.0,M-LC,10.67,171732.0,30892.0,13601.0,-0.33,21.93,7.92,31.59,66.0,2.27,1.21,54.23,XY24,NTT,BANKS
85,WHIRLPOOL,2270.00,-40.03,54.0,M-SC,5.35,124200.0,9402.0,84642.0,-1.78,8.19,68.15,81.92,223.0,0.11,0.87,47.03,XR,NTT,DURABLES
36,INDIAMART,4810.62,-52.96,57.0,H-SC,5.86,131212.0,7876.0,123746.0,0.30,6.39,94.31,106.72,119.0,0.06,0.92,31.26,AR,ATH,MISC


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
85,WHIRLPOOL,2270.00,-40.03,54.0,M-SC,5.35,124200.0,9402.0,84642.0,-1.78,8.19,68.15,81.92,223.0,0.11,0.87,47.03,XR,NTT,DURABLES
46,KPIGREEN,731.05,8.64,58.0,H-SC,7.39,129490.0,4193.0,54735.0,-3.42,3.35,42.27,47.03,141.0,0.08,0.91,62.13,MH,ATH,POWER
36,INDIAMART,4810.62,-52.96,57.0,H-SC,5.86,131212.0,7876.0,123746.0,0.30,6.39,94.31,106.72,119.0,0.06,0.92,31.26,AR,ATH,MISC
31,HINDZINC,730.22,24.68,43.0,M-LC,10.43,203493.0,-1583.0,114160.0,-1.17,-0.77,56.10,54.89,52.0,-0.01,1.43,23.14,X5K,ATH,METALS
86,WIPRO,420.00,-15.15,44.0,M-LC,5.47,149023.0,-1922.0,111380.0,1.03,-1.27,74.74,72.51,53.0,-0.02,1.05,4.67,XR,NTT,IT


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TATAMOTORS,1065.00,-53.17,21.0,X-LC,28.48,147560.0,-128963.0,239032.0,-2.52,-46.64,161.99,39.81,54.0,-0.54,1.04,4.00,XY24,NTT,AUTO
12,BATAINDIA,2096.00,-41.58,25.0,X-SC,6.95,87274.0,-41396.0,86689.0,-1.37,-32.17,99.33,35.20,219.0,-0.48,0.61,0.17,X40,NTT,FOOTWEAR
44,JSWINFRA,342.00,-23.75,28.0,X-MC,4.69,191565.0,-8782.0,39290.0,-2.04,-4.38,20.51,15.23,178.0,-0.22,1.35,26.45,X40N,NTT,REALTY
23,DMART,5391.45,-19.92,29.0,X-LC,8.28,204150.0,-6257.0,65430.0,-2.38,-2.97,32.05,28.12,38.0,-0.10,1.44,19.95,X40N,ATH,FMCG
84,VOLTAS,1530.00,-6.95,30.0,X-MC,4.46,195315.0,3573.0,34180.0,-4.22,1.86,17.50,19.69,99.0,0.10,1.37,8.79,XY25,NTT,AC


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
52,PGHH,17907.65,-31.34,35.0,X-MC,0.91,198720.0,-2100.0,69890.0,-1.14,-1.05,35.17,33.76,80.0,-0.03,1.40,4.10,X40,ATH,FMCG
41,ITC,452.00,-38.34,46.0,X-LC,1.13,199488.0,-650.0,21545.0,-0.23,-0.32,10.80,10.44,4.0,-0.03,1.40,5.22,X40,NTT,FMCG
66,SIEMENS,4671.50,-3.90,43.0,H-LC,1.24,154300.0,-31795.0,79279.0,-1.08,-17.09,51.38,25.51,15.0,-0.40,1.09,13.71,AR,ATH,ELECTRICAL
28,HAVELLS,2069.16,-8.73,35.0,X-MC,3.08,304374.0,-23617.0,130150.0,-2.49,-7.20,42.76,32.48,92.0,-0.18,2.14,3.40,X40,ATH,ELECTRICAL
80,UNITDSPR,1644.00,-6.95,62.0,X-LC,4.13,243844.0,9700.0,38918.0,-2.29,4.14,15.96,20.77,86.0,0.25,1.72,11.35,X40N,NTT,BREWERIES


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,QUESS,986.00,-50.96,32.0,X-SC,37.76,48951.0,-16055.0,166982.0,-2.76,-24.70,341.12,232.18,198.0,-0.10,0.34,0.00,XY24,NTT,MISC
12,BATAINDIA,2096.00,-41.58,25.0,X-SC,6.95,87274.0,-41396.0,86689.0,-1.37,-32.17,99.33,35.20,219.0,-0.48,0.61,0.17,X40,NTT,FOOTWEAR
51,PAGEIND,51769.93,-29.36,34.0,X-MC,7.23,79840.0,-3670.0,23697.0,-0.54,-4.39,29.68,23.98,82.0,-0.15,0.56,1.17,X40,ATH,APPARELS
20,COLPAL,3726.84,-6.52,42.0,X-MC,7.33,218290.0,-45075.0,154397.0,-0.36,-17.12,70.73,41.51,84.0,-0.29,1.54,2.44,XY25,ATH,FMCG
3,ACC,3906.00,-38.50,43.0,X-MC,2.45,183070.0,-54881.0,207528.0,-0.10,-23.06,113.36,64.15,174.0,-0.26,1.29,2.69,XY24,BTT,CEMENT


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,QUESS,986.00,-50.96,32.0,X-SC,37.76,48951.0,-16055.0,166982.0,-2.76,-24.70,341.12,232.18,198.0,-0.10,0.34,0.00,XY24,NTT,MISC
51,PAGEIND,51769.93,-29.36,34.0,X-MC,7.23,79840.0,-3670.0,23697.0,-0.54,-4.39,29.68,23.98,82.0,-0.15,0.56,1.17,X40,ATH,APPARELS
57,RELAXO,1176.00,-39.68,45.0,X-SC,4.28,82184.0,-62976.0,138907.0,0.91,-43.38,169.02,52.31,136.0,-0.45,0.58,10.01,X40N,NTT,FOOTWEAR
12,BATAINDIA,2096.00,-41.58,25.0,X-SC,6.95,87274.0,-41396.0,86689.0,-1.37,-32.17,99.33,35.20,219.0,-0.48,0.61,0.17,X40,NTT,FOOTWEAR
2,ABBOTINDIA,35195.00,-10.45,43.0,X-MC,2.60,88530.0,-2148.0,17051.0,0.51,-2.37,19.26,16.44,101.0,-0.13,0.62,17.33,X40,ATH,PHARMA


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
75,TCS,4389.97,-29.01,49.0,X-LC,9.87,283297.0,-62659.0,129353.0,0.79,-18.11,45.66,19.28,1.0,-0.48,1.99,4.73,X40,ATH,IT
39,INFY,2275.00,-19.60,49.0,X-LC,5.13,313131.0,243.0,171439.0,0.15,0.08,54.75,54.87,3.0,0.00,2.20,8.37,X40,BTT,IT
41,ITC,452.00,-38.34,46.0,X-LC,1.13,199488.0,-650.0,21545.0,-0.23,-0.32,10.80,10.44,4.0,-0.03,1.40,5.22,X40,NTT,FMCG
83,VBL,671.64,-16.43,54.0,X-LC,6.87,301104.0,-14738.0,127397.0,0.20,-4.67,42.31,35.67,5.0,-0.12,2.12,8.54,X40N,ATH,FMCG
1,ABB,7934.00,-39.67,53.0,H-LC,5.06,252288.0,-9331.0,128541.0,0.23,-3.57,50.95,45.57,7.0,-0.07,1.78,7.35,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
19,COFFEEDAY,80.00,-49.70,35.0,L-SC,27.74,74880.0,-38669.0,78721.0,-3.63,-34.05,105.13,35.27,268.0,-0.49,0.53,83.27,XR,NTT,HOTELS
7,ASIANTILES,137.00,7400.00,66.0,L-SC,10.60,82278.0,-11532.0,88153.0,-1.50,-12.29,107.14,81.67,269.0,-0.13,0.58,59.34,XR,NTT,CERAMICS
49,MASFIN,398.61,-19.60,48.0,H-SC,8.35,91410.0,-6570.0,28173.0,-2.59,-6.71,30.82,22.05,152.0,-0.23,0.64,32.87,XR,ATH,FINANCE
85,WHIRLPOOL,2270.00,-40.03,54.0,M-SC,5.35,124200.0,9402.0,84642.0,-1.78,8.19,68.15,81.92,223.0,0.11,0.87,47.03,XR,NTT,DURABLES
46,KPIGREEN,731.05,8.64,58.0,H-SC,7.39,129490.0,4193.0,54735.0,-3.42,3.35,42.27,47.03,141.0,0.08,0.91,62.13,MH,ATH,POWER


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
59,REPCOHOME,880.00,-53.06,66.0,H-SC,7.29,269572.0,-16933.0,274263.0,-3.11,-5.91,101.74,89.82,134.0,-0.06,1.90,42.62,XY24,NTT,FINANCE
81,VAIBHAVGBL,521.00,73.55,63.0,H-SC,2.29,150250.0,-32525.0,143594.0,-2.97,-17.80,95.57,60.77,125.0,-0.23,1.06,34.57,XR,NTT,JEWELLERY
68,SONACOMS,806.63,-32.06,66.0,M-SC,6.91,84595.0,-16564.0,56569.0,-0.08,-16.37,66.87,39.54,202.0,-0.29,0.60,19.29,AR,ATH,AUTO
62,SAMMAANCAP,326.00,-196.59,69.0,M-SC,33.06,167112.0,17892.0,126287.0,-0.29,11.99,75.57,96.62,208.0,0.14,1.18,80.85,XY25,NTT,FINANCE
46,KPIGREEN,731.05,8.64,58.0,H-SC,7.39,129490.0,4193.0,54735.0,-3.42,3.35,42.27,47.03,141.0,0.08,0.91,62.13,MH,ATH,POWER


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.74
1,25,44.43
2,50,75.50


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    43.77
LC    33.20
MC    23.02
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     29.96
X40      16.22
X40N     12.41
XY25     11.92
XR        9.89
AR        7.86
OX40N     6.55
X200      1.79
X5K       1.43
SR        1.05
MH        0.91
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    24.29
X-LC    22.47
X-MC    18.71
M-SC    12.57
X-SC     5.47
M-LC     5.10
H-LC     4.66
H-MC     2.32
M-MC     1.67
L-SC     1.44
L-LC     0.97
L-MC     0.32
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,15.34,-3.87,37.45
IT,12.45,-18.72,82.32
FINANCE,12.15,-9.63,59.33
MISC,7.72,-18.33,78.02
ELECTRICAL,5.77,-10.27,49.36
PAINTS,5.59,-16.13,33.44
BANKS,4.05,-17.78,86.30
INSURANCE,3.86,-0.81,38.40
AUTO,2.73,-48.36,110.28


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3317344.0,22
XR,1254326.0,13
AR,1171086.0,9
X40,848274.0,12
X40N,706264.0,9
OX40N,552899.0,9
XY25,480282.0,8
SR,272286.0,2
X5K,114160.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3381567.0,24
M-SC,1257489.0,15
X-MC,1214901.0,14
X-LC,1086635.0,13
X-SC,570014.0,6
M-LC,284341.0,4
H-LC,277059.0,3
H-MC,258486.0,2
L-SC,256801.0,3


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1188060.0      6
           AR         839306.0      5
           XR         784502.0      7
M-SC       XY24       727596.0      6
X-MC       XY24       566872.0      4
X-LC       X40        460309.0      5
X-SC       X40N       316343.0      4
X-LC       XY24       315365.0      2
X-MC       X40        301276.0      6
H-SC       SR         272286.0      2
           OX40N      242678.0      3
X-LC       X40N       231745.0      3
H-LC       AR         207820.0      2
X-MC       XY25       188577.0      2
H-MC       XY24       184042.0      1
X-SC       XY24       166982.0      1
L-SC       XR         166874.0      2
X-MC       X40N       158176.0      2
M-MC       XY24       154826.0      1
M-SC       OX40N      145850.0      4
           XR         133796.0      2
           XY25       126287.0      1
           AR         123960.0      2
M-LC       X5K        114160.0      1
           XR         111380.0      1
L-SC       OX40N       89927.0      1
X-SC       X40         86689.0      1
X-LC       XY25        79216.0      3
H-MC       OX40N       74444.0      1
H-LC       X200        69239.0      1
L-MC       XR          57774.0      1
H-SC       MH          54735.0      1
M-LC       XY25        45200.0      1
L-LC       XY25        41002.0      1
M-LC       XY24        13601.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 35.0 seconds
